In [1]:
import os 
os.chdir('../')
%pwd 

'/Users/yashasuttangi/Developer/MachineLearning/Projects/CancerClassification/Cancer-classification-end-to-end-using-MLFlow-and-DVC'

In [2]:
import dagshub
dagshub.init(
    repo_owner='yashuttangi', 
    repo_name='Cancer-classification-end-to-end-using-MLFlow-and-DVC', 
    mlflow=True
)

Accessing as yashuttangi

Initialized MLflow to track repo "yashuttangi/Cancer-classification-end-to-end-using-MLFlow-and-DVC"

Repository yashuttangi/Cancer-classification-end-to-end-using-MLFlow-and-DVC initialized!

In [3]:
import tensorflow as tf 
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path 
    training_data: Path 
    all_params: dict 
    params_image_size: list 
    params_batch_size: int 

In [5]:
from ChestCancerClassifier.constants import * 
from ChestCancerClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/ChestCancerClassificationDataset",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [14]:
import mlflow 
import mlflow.keras
from urllib.parse import urlparse 

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config 
    
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split = 0.30 
        )

        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            batch_size = self.config.params_batch_size,
            interpolation = "bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
    
    def log_into_mlflow(self):
        with mlflow.start_run(run_name = 'model_evaluation'):
            # log params
            mlflow.log_params(self.config.all_params)

            # log metrics 
            mlflow.log_metrics({
                "loss": self.score[0],
                "accuracy": self.score[1]
            })

            # load_model
            mlflow.keras.log_model(
                self.model,
                artifact_path="model",
                registered_model_name="VGG16Model"
            )

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e 


Reading yaml file : config/config.yaml
[2026-01-29 15:12:07,540: INFO: common]: YAML file: config/config.yaml loaded successfully
Reading yaml file : params.yaml
[2026-01-29 15:12:07,542: INFO: common]: YAML file: params.yaml loaded successfully
[2026-01-29 15:12:07,542: INFO: common]: created directory at : artifacts
Found 298 images belonging to 3 classes.


2026-01-29 15:12:07.720281: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


19/19 [==============================] - 12s 644ms/step - loss: 2.1459 - accuracy: 0.8893
[2026-01-29 15:12:20,193: INFO: common]: JSON file saved at: {path}


2026/01/29 15:12:21 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-01-29 15:12:21,610: WARNING: save]: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: /var/folders/m_/20v0dlvj0q525r1dgql5nqgc0000gn/T/tmpx2ui1xcx/model/data/model/assets
[2026-01-29 15:12:21,776: INFO: builder_impl]: Assets written to: /var/folders/m_/20v0dlvj0q525r1dgql5nqgc0000gn/T/tmpx2ui1xcx/model/data/model/assets


2026/01/29 15:12:25 WARNING mlflow.utils.requirements_utils: The following packages were not found in the public PyPI package index as of 2023-02-28; if these packages are not present in the public PyPI index, you must install them manually before loading your model: {'backports-tarfile'}
/Users/yashasuttangi/Developer/MachineLearning/Projects/CancerClassification/Cancer-classification-end-to-end-using-MLFlow-and-DVC/myenv/lib/python3.11/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2026/01/29 15:12:25 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'VGG16Model'.
2026/01/29 15:12